In [66]:
# autoreload reloads modules automatically before entering the execution of code typed at the IPython prompt.
%reload_ext autoreload
%autoreload 2
%aimport feature_preprocessing_pipeline

In [116]:
from sklearn import set_config
from os import system
from os.path import join, isfile
from joblib import dump, load
from feature_preprocessing_pipeline import categorize, extract_features, clean_variables, evaluate_prediction
from sklearn import tree
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import tree
#from dtreeviz.trees import dtreeviz

In [57]:
set_config(display='diagram') 

In [58]:
dataset_directory_path = 'dataset/'
model_directory_path = 'model/'

In [90]:
model_name = 'RFR_1829'
model_filename = f'{model_name}.joblib'
model_path = join(model_directory_path, model_filename)
model = load(model_path)

In [60]:
def get_data(file_path, target_column,dataset_directory_path='./dataset/', sample=None, callback=None):
    data = pd.read_csv(join(dataset_directory_path, file_path), index_col=0)
    if callback:
        data = callback(data)
    if sample:
        data = data.sample(n=sample)
    X = data.drop(labels=[target_column], axis=1)
    y = np.log(data[target_column])
    return X,y



In [82]:
X_test = pd.read_csv(join(dataset_directory_path, 'X_test.csv'), index_col=0)
y_test = pd.read_csv(join(dataset_directory_path, 'y_test.csv'), index_col=0)

## Description du model 

### Diagramme

In [61]:
model

Pipeline(steps=[('features_extraction',
                 FunctionTransformer(func=<function extract_features at 0x7fa4c038a160>)),
                ('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('poly',
                                                  PolynomialFeatures(include_bias=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa4c017da60>),
                                                 ('mpg_discretizer',
                                                  KBinsDiscretizer(n_bin...
                                                                   OneHotEncoder(handle_unknown='ignore'))],
                                                           verbose=True),
                                                  ['transmission']),
                                                 ('fuel_type_pipe',
                                                  Pipeline(steps=[('OHE',
                                                                   OneHotEncoder(handle_unknown='ignore'))],
                                                           verbose=True),
                                                  ['fuel_type'])],
                                   verbose=2)),
                ('scaler', StandardScaler(with_mean=False)),
                ('random_forest',
                 RandomForestRegressor(max_depth=50, min_samples_split=6,
                                       n_jobs=-1, verbose=True,
                                       warm_start=True))],
         verbose=True)

In [ ]:
#model.fit(X_test,y_test.values.ravel())
model.fit(X_test,y_test.values.ravel())
rf = model[3]#.estimators_[1]
#print(tree.feature_importances_)

_ = tree.plot_tree(rf.estimators_[0], filled=True)


Extract Features
[Pipeline]  (step 1 of 4) Processing features_extraction, total=   0.0s
[ColumnTransformer] .......... (1 of 9) Processing poly, total=   0.0s
[ColumnTransformer]  (2 of 9) Processing mpg_discretizer, total=   0.0s
[ColumnTransformer]  (3 of 9) Processing tax_discretizer, total=   0.0s
[ColumnTransformer]  (4 of 9) Processing engine_size_discretizer, total=   0.0s
[Pipeline] ........ (step 1 of 1) Processing discretize, total=   0.0s
[ColumnTransformer] ..... (5 of 9) Processing year_pipe, total=   0.0s
[Pipeline] ............... (step 1 of 1) Processing OHE, total=   0.0s
[ColumnTransformer] .... (6 of 9) Processing model_pipe, total=   0.0s
[Pipeline] ............... (step 1 of 1) Processing OHE, total=   0.0s
[ColumnTransformer] .... (7 of 9) Processing brand_pipe, total=   0.0s
[Pipeline] ............... (step 1 of 1) Processing OHE, total=   0.0s
[ColumnTransformer]  (8 of 9) Processing transmission_pipe, total=   0.0s
[Pipeline] ............... (step 1 of 1) Pro

/home/ghostbot/anaconda3/envs/tf_cpu/lib/python3.8/site-packages/sklearn/preprocessing/_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/ghostbot/anaconda3/envs/tf_cpu/lib/python3.8/site-packages/sklearn/preprocessing/_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '


[Pipeline] ............ (step 3 of 4) Processing scaler, total=   0.1s
[Pipeline] ..... (step 4 of 4) Processing random_forest, total=   0.0s


/home/ghostbot/anaconda3/envs/tf_cpu/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:369: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


### Variables

In [63]:
rfr = model[3]
print(f"Le model {rfr} a {rfr.n_features_in_} variables")

Le model RandomForestRegressor(max_depth=50, min_samples_split=6, n_jobs=-1,
                      verbose=True, warm_start=True) a 272 variables


### Durée d'entraînement

## Performance

### Learning Curve

![Learning curve](./Learning_curve_model_677.png)

Le premier graphique montre la courbe d'apprentissage.

Le deuxième graphique montre le temps nécessaire aux modèles pour s'entraîner avec différentes tailles de données d'entraînement. 

Le troisième graphique montre le temps nécessaire à l'entraînement des modèles pour chaque taille de données d'entraînement.

In [64]:
print(f"Test set:\n X {X_test.shape} y {y_test.shape}")


Test set:
 X (7824, 9) y (7824, 1)


In [80]:
evaluate_prediction(model, X_test, y_test,sample=25)


Extract Features
RMSE: 1529.201266839476


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


MODEL PREDICTION TESTING ────────────────────────────────────────────────────────────────────

Prediction results                              
┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Prediction ┃ Real Price ┃ Error ┃ Percentage ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│    7766    │    7341    │  425  │    6 %     │
│   15606    │   16000    │  394  │    2 %     │
│   11309    │   10997    │  312  │    3 %     │
│   19608    │   19279    │  329  │    2 %     │
│   11108    │   10999    │  109  │    1 %     │
│   19534    │   17894    │ 1640  │    9 %     │
│   14757    │   14488    │  269  │    2 %     │
│   42085    │   41630    │  455  │    1 %     │
│    6681    │    7499    │  818  │    11 %    │
│    7906    │    8995    │ 1089  │    12 %    │
│    6218    │    5998    │  220  │    4 %     │
│   13150    │   13494    │  344  │    3 %     │
│   10682    │   12500    │ 1818  │    15 %    │
│   14905    │   13599    │ 1306  │    10 %    │
│    8628    │    8740    │  112  │    1 %     │
│   38590    │   34950    │ 3640  │    10 %    │
│   11916    │   12700    │  784  │    6 %     │
│   12977    │   13494    │  517  │    4 %     │
│   10350    │   14591    │ 4241  │    29 %    │
│   20696    │   20500    │  196  │    1 %     │
│   16640    │   16497    │  143  │    1 %     │
│   41711    │   45490    │ 3779  │    8 %     │
│    8135    │    6994    │ 1141  │    16 %    │
│   16675    │   16297    │  378  │    2 %     │
│   22166    │   22095    │  71   │    0 %     │
└────────────┴────────────┴───────┴────────────┘

In [13]:
#tree.plot_tree(model[2].base_estimator_, 
                   #feature_names=data.feature_names,  
                   #class_names=data.target_names,
#                   filled=True)

## Context d'application

Décrire le contexte d'évaluation du model. cdts dans lesquelles la précision est valable.

### Limites

Décrire les limites dans lesquelles le modèle est utilisable.

## Analyse et choix

## Méthode